<a href="https://colab.research.google.com/github/vinods03/deep_learning/blob/main/NLP_13_Spotify_Reviews_using_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
sia = SentimentIntensityAnalyzer()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/spotify_reviews.csv')

In [ ]:
df.head()

,Time_submitted,Review,Rating,Total_thumbsup,Reply
0,2022-07-09 15:00:00,"Great music service, the audio is high quality...",5,2,NaN
1,2022-07-09 14:21:22,Please ignore previous negative rating. This a...,5,1,NaN
2,2022-07-09 13:27:32,"This pop-up ""Get the best Spotify experience o...",4,0,NaN
3,2022-07-09 13:26:45,Really buggy and terrible to use as of recently,1,1,NaN
4,2022-07-09 13:20:49,Dear Spotify why do I get songs that I didn't ...,1,1,NaN


In [ ]:
df = df[['Review','Total_thumbsup','Rating']]

In [ ]:
df.head(5)

,Review,Total_thumbsup,Rating
0,"Great music service, the audio is high quality...",2,5
1,Please ignore previous negative rating. This a...,1,5
2,"This pop-up ""Get the best Spotify experience o...",0,4
3,Really buggy and terrible to use as of recently,1,1
4,Dear Spotify why do I get songs that I didn't ...,1,1


In [ ]:
df['Rating'].value_counts()

5    22095
1    17653
4     7842
2     7118
3     6886
Name: Rating, dtype: int64

In [ ]:
df.isnull().sum()

Review            0
Total_thumbsup    0
Rating            0
dtype: int64

In [ ]:
# not needed
# df.dropna(inplace = True)

In [ ]:
# for i in df.itertuples():
#   print(i)

blanks = []

for i, rv, thumbsup, lb in df.itertuples():
  if rv.isspace():
    blanks = blanks.append(i)

blanks

[]

In [ ]:
# not needed
# df.drop(blanks, inplace = True)

In [ ]:
df['Rating'].replace(1, value = 'Negative', inplace = True)
df['Rating'].replace(2, value = 'Negative', inplace = True)
df['Rating'].replace(3, value = 'Negative', inplace = True)
df['Rating'].replace(4, value = 'Positive', inplace = True)
df['Rating'].replace(5, value = 'Positive', inplace = True)

In [ ]:
df['scores'] = df['Review'].apply(lambda x: sia.polarity_scores(x))

In [ ]:
df.head(5)

,Review,Total_thumbsup,Rating,scores
0,"Great music service, the audio is high quality...",2,Positive,"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'comp..."
1,Please ignore previous negative rating. This a...,1,Positive,"{'neg': 0.243, 'neu': 0.353, 'pos': 0.404, 'co..."
2,"This pop-up ""Get the best Spotify experience o...",0,Positive,"{'neg': 0.107, 'neu': 0.635, 'pos': 0.258, 'co..."
3,Really buggy and terrible to use as of recently,1,Negative,"{'neg': 0.296, 'neu': 0.704, 'pos': 0.0, 'comp..."
4,Dear Spotify why do I get songs that I didn't ...,1,Negative,"{'neg': 0.0, 'neu': 0.74, 'pos': 0.26, 'compou..."


In [ ]:
df['compound'] = df['scores'].apply(lambda x: x['compound'])

In [ ]:
df.head(5)

,Review,Total_thumbsup,Rating,scores,compound
0,"Great music service, the audio is high quality...",2,Positive,"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'comp...",0.9211
1,Please ignore previous negative rating. This a...,1,Positive,"{'neg': 0.243, 'neu': 0.353, 'pos': 0.404, 'co...",0.6249
2,"This pop-up ""Get the best Spotify experience o...",0,Positive,"{'neg': 0.107, 'neu': 0.635, 'pos': 0.258, 'co...",0.5859
3,Really buggy and terrible to use as of recently,1,Negative,"{'neg': 0.296, 'neu': 0.704, 'pos': 0.0, 'comp...",-0.5209
4,Dear Spotify why do I get songs that I didn't ...,1,Negative,"{'neg': 0.0, 'neu': 0.74, 'pos': 0.26, 'compou...",0.7149


In [ ]:
df['review_type'] = df['compound'].apply(lambda x: 'Positive' if x > 0 else 'Negative')

In [ ]:
df.head()

,Review,Total_thumbsup,Rating,scores,compound,review_type
0,"Great music service, the audio is high quality...",2,Positive,"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'comp...",0.9211,Positive
1,Please ignore previous negative rating. This a...,1,Positive,"{'neg': 0.243, 'neu': 0.353, 'pos': 0.404, 'co...",0.6249,Positive
2,"This pop-up ""Get the best Spotify experience o...",0,Positive,"{'neg': 0.107, 'neu': 0.635, 'pos': 0.258, 'co...",0.5859,Positive
3,Really buggy and terrible to use as of recently,1,Negative,"{'neg': 0.296, 'neu': 0.704, 'pos': 0.0, 'comp...",-0.5209,Negative
4,Dear Spotify why do I get songs that I didn't ...,1,Negative,"{'neg': 0.0, 'neu': 0.74, 'pos': 0.26, 'compou...",0.7149,Positive


In [ ]:
df.to_csv('/content/drive/MyDrive/processed_spotify_reviews.csv')

In [ ]:
import sklearn.metrics as metrics
accuracy = metrics.accuracy_score(df['Rating'], df['review_type'])
error = 1 - accuracy
precision = metrics.precision_score(df['Rating'], df['review_type'], pos_label = 'Negative')
recall = metrics.recall_score(df['Rating'], df['review_type'], pos_label = 'Negative')
f1 = metrics.f1_score(df['Rating'], df['review_type'], pos_label = 'Negative')
cm = metrics.confusion_matrix(df['Rating'], df['review_type'])

print('accuracy:', accuracy)
print('error:', error)
print('precision:', precision)
print('recall:', recall)
print('f1:',f1)
print('\n')
print(cm)

from sklearn.metrics import classification_report
print('\n')
print(classification_report(df['Rating'], df['review_type']))

accuracy: 0.7111406955222911
error: 0.2888593044777089
precision: 0.8216788084079625
recall: 0.5593707552831917
f1: 0.6656141933543828


[[17708 13949]
 [ 3843 26094]]


              precision    recall  f1-score   support

    Negative       0.82      0.56      0.67     31657
    Positive       0.65      0.87      0.75     29937

    accuracy                           0.71     61594
   macro avg       0.74      0.72      0.71     61594
weighted avg       0.74      0.71      0.70     61594

